In [29]:
import glob
import pandas as pd
import nibabel as nb
from nistats.second_level_model import SecondLevelModel

In [7]:
networks = ['CER','LIM','MOT','VIS','DMN','FP','VATT_SAL']

nets_rmaps = {}

for net in networks:
    connectivity_maps = glob.glob("rest/rmap_seeds/*sub*_"+net+".nii.gz")
    rmap_list = []
    for rmap in connectivity_maps:
        rmap_list.append(rmap)
    nets_rmaps[net] = rmap_list
        

In [26]:
nets_rmaps.keys()

dict_keys(['CER', 'LIM', 'MOT', 'VIS', 'DMN', 'FP', 'VATT_SAL'])

In [17]:
network_interest = "DMN"

In [18]:
cimaq_diagnostic = '../cimaq_csv/cimaq_diagnostic_Nov_2019.csv'
cimaq_diagnostic = pd.read_csv(cimaq_diagnostic)
print(cimaq_diagnostic.columns)

Index(['Identifiers',
       'V01 36448_diagnostic_clinique,36448_diagnostic_clinique',
       'V07 50658_diagnostic_clinique_2ans,50658_diagnostic_clinique',
       'V01 demographics,CandID'],
      dtype='object')


In [19]:
print(set(cimaq_diagnostic['V01 36448_diagnostic_clinique,36448_diagnostic_clinique'].values))
cimaq_diagnostic = cimaq_diagnostic[['V01 demographics,CandID','V01 36448_diagnostic_clinique,36448_diagnostic_clinique']]
cimaq_diagnostic.rename(columns = {"V01 demographics,CandID":"subID","V01 36448_diagnostic_clinique,36448_diagnostic_clinique":"diagnostic" },inplace=True)
cimaq_diagnostic.head()

{'trouble_cognitif_léger_tardif', 'trouble_cognitif_léger_précoce', 'troubles_subjectifs_de_cognition', 'cognitivement_sain_(cs)', 'démence_de_type_alzheimer-légère'}


,subID,diagnostic
0,327986,troubles_subjectifs_de_cognition
1,934539,démence_de_type_alzheimer-légère
2,368387,troubles_subjectifs_de_cognition
3,658178,troubles_subjectifs_de_cognition
4,641853,troubles_subjectifs_de_cognition


In [20]:
cimaq_diagnostic["SCD"] = 0
cimaq_diagnostic["CN"] = 0
subjectID_set = set([sub.split("sub")[1].split("_")[0] for sub in nets_rmaps["CER"]])
for i,r in cimaq_diagnostic.iterrows():
    
    if(str(r["subID"]) in subjectID_set):
        if(r["diagnostic"] == "troubles_subjectifs_de_cognition"):
            cimaq_diagnostic.loc[i,"SCD"] = 1
        if(r["diagnostic"] == "cognitivement_sain_(cs)"):
            cimaq_diagnostic.loc[i,"CN"] = 1
        cimaq_diagnostic.loc[i,"subID"] = str(r["subID"])

print(np.sum(cimaq_diagnostic["CN"]))
print(np.sum(cimaq_diagnostic["SCD"]))

23
54


In [21]:
cimaq_diagnostic = cimaq_diagnostic.loc[(cimaq_diagnostic["SCD"] != 0) | (cimaq_diagnostic["CN"] != 0)]

In [22]:
cimaq_diagnostic

,subID,diagnostic,SCD,CN
3,658178,troubles_subjectifs_de_cognition,1,0
7,920577,troubles_subjectifs_de_cognition,1,0
11,254530,troubles_subjectifs_de_cognition,1,0
20,956130,troubles_subjectifs_de_cognition,1,0
22,878354,troubles_subjectifs_de_cognition,1,0
...,...,...,...,...
289,437101,cognitivement_sain_(cs),0,1
293,968913,troubles_subjectifs_de_cognition,1,0
296,785245,troubles_subjectifs_de_cognition,1,0
299,778749,troubles_subjectifs_de_cognition,1,0


In [27]:
design_matrix = cimaq_diagnostic[["CN","SCD"]]
design_matrix

,CN,SCD
3,0,1
7,0,1
11,0,1
20,0,1
22,0,1
...,...,...
289,1,0
293,0,1
296,0,1
299,0,1


In [25]:
second_level_model = SecondLevelModel().fit(
    nets_rmaps["CER"], design_matrix=design_matrix)

In [28]:
second_level_model

SecondLevelModel(mask_img=None, memory=Memory(location=None), memory_level=1,
                 minimize_memory=True, n_jobs=1, smoothing_fwhm=None,
                 verbose=0)